<a href="https://colab.research.google.com/github/EJHyun/Noc_Lab/blob/master/SGD_SVM_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
import cupy as cp
from sklearn import metrics
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from math import ceil, sqrt
import math
import copy
from cupy.lib import stride_tricks

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

import argparse
import os
import struct

In [0]:
class BinarySVC(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1, eta=0.001, batch_size=1, max_epoch=100, random_state=1, pos_=0, mth = 40):
        self.lambda_ = 1./C
        self.eta = eta
        self.batch_size = batch_size
        self.max_epoch = max_epoch
        self.random_state = random_state
        self.pos_ = pos_
        self.mth = mth

    def hinge_losses(self, x, y):
        c = 1./self.lambda_
        w_sum = 0
        losses = 0
        W = []
        x_len = len(x)
        
        inner_ = cp.dot(x,self.w_)
        mult_Y = cp.multiply(y,(inner_ + self.b_))
        one_ = cp.ones(x_len)
        one_ -= mult_Y
        
        check_max = cp.greater(one_, 0) 
        
        Loss = cp.multiply(one_, check_max)
        losses = cp.sum(Loss)

        losses /= x_len
        
        w_sum = cp.sum(cp.multiply(self.w_,self.w_))

        w_sum /= (2*c)
        losses += w_sum
        return losses

    def check_min(self, hinge, h_min, isFirst, e):
        if(hinge < h_min[0] or isFirst):
            h_min[0] = hinge
            h_min[1] = self.w_
            h_min[2] = self.b_
            h_min[3] = e
        return self

    def plot_loss(self, losses, h_min):
    #     plt.figure(figsize=(10,10))
    #     plot_name = '{} vs Rest'.format(self.pos_) 
    #     plt.title(plot_name, size = 15)
    #     plt.plot(losses)
    #     plt.ylabel('Loss', size = 15)
    #     plt.xlabel('Iteration', size = 15)
    #     plt.show()
          print("class",self.pos_,"minimum loss = ", h_min[0], "at", h_min[3], "th update")
          return self

    def fit(self, X, y=None):
        X = cp.asarray(X)
        y = cp.asarray(y)
        r_gen = np.random.RandomState(self.random_state)
        self.w_ = r_gen.normal(loc=0.0, scale=0.1, size=1 + X.shape[1])
        self.b_, self.w_ = self.w_[0], self.w_[1:]
        self.w_ = cp.asarray(self.w_)

        losses=[]       #for loss plot
        h_min = [0,0,0,0]
        eta = 0.0
        n_batches = ceil(X.shape[0] / self.batch_size)
        Size = self.max_epoch * n_batches
        Mth = Size * self.mth / 100
        update_ctr = 1

        for e in range(self.max_epoch):
            idx = [i for i in range(X.shape[0])]
            r_gen.shuffle(idx)

            for j in range(n_batches):

                if(update_ctr - 1 < Mth):
                    eta += self.eta / Mth
                else:
                    eta = (1 + math.cos((update_ctr) * math.pi / Size)) * self.eta / 2
                   
                batch_idx = idx[self.batch_size * j : self.batch_size * (j+1)]
                grad_w, grad_b = self._get_gradient(X[batch_idx], y[batch_idx])
                self.w_ = self.w_ - 1 * self.eta * grad_w
                self.b_ = self.b_ - 1 * self.eta * grad_b

                n = int(Size * 990/1000) # check latter iter
                #n = 100
                if(update_ctr >= n):
                    hinge = self.hinge_losses(X, y)      # calculate hinge loss
                    losses.append(hinge)                 # append for plot
                    self.check_min(hinge, h_min, update_ctr==n, update_ctr)# update h_min if minimum or First
                update_ctr += 1
            
        self.w_ = h_min[1]
        self.b_ = h_min[2]
        
        self.plot_loss(losses, h_min)

        return self

    def _get_gradient(self, X_batch, y_batch):
        grad_w = cp.zeros(X_batch.shape[1])
        grad_b = 0
        mask = cp.less_equal(cp.multiply(y_batch, cp.dot(X_batch, self.w_))+self.b_, 1)
        Xy = cp.multiply(X_batch.T, y_batch)
        masked_Xy = cp.multiply(Xy, mask)
        grad_w = cp.sum(-masked_Xy, axis=1)
        grad_w = grad_w / self.batch_size + self.lambda_ * self.w_

        masked_y = cp.multiply(y_batch, mask)
        grad_b = cp.sum(-masked_y, axis=0)
        grad_b = grad_b / self.batch_size

        return grad_w, grad_b

class SVC(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1, eta=0.001, batch_size=1, max_epoch=100, random_state=1, mth = 40):
        self.C = C
        self.eta = eta
        self.batch_size = batch_size
        self.max_epoch = max_epoch
        self.random_state = random_state
        self.mth = mth

    def fit(self, X, y=None):
        X = cp.asarray(X)
        y = cp.asarray(y)
        self.classes_ = cp.unique(y)
        self.clfs_per_classs_ = []
        # print('[fit]')
        for pos in self.classes_:
            # print(f"[{pos}]", end='')
            Y = cp.where(y == pos, 1, -1)
            self.clfs_per_classs_.append(BinarySVC(self.C, self.eta, self.batch_size, self.max_epoch, self.random_state, pos_ = pos, mth = self.mth).fit(X, Y))
        self.w_ = [clf.w_ for clf in self.clfs_per_classs_]
        self.b_ = [clf.b_ for clf in self.clfs_per_classs_]
        return self

    def get_weights(self):
        return self.w_, self.b_

    def predict(self, X):
        # print('\n[predict]')
        X = cp.asarray(X)
        self.w_ = cp.stack(self.w_)
        self.b_ = cp.stack(self.b_)
        hyper_planes = cp.dot(X, self.w_.T) + self.b_
        predictions = self.classes_[cp.argmax(hyper_planes, axis=1)]

        return cp.asnumpy(predictions)

In [0]:
def img_generator(img):
    with open(os.path.join(img), 'rb') as fimg:
        _, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(num, rows * cols)

    def get_img(idx): return img[idx]

    for i in range(num):
        yield get_img(i)

def lbl_generator(label):
    with open(os.path.join(label), 'rb') as flbl:
        _, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    def get_label(idx): return lbl[idx]

    for i in range(num):
        yield get_label(i)

def get_data(train_image_path, train_label_path):
    x_d1 = img_generator(train_image_path)
    y_d1 = lbl_generator(train_label_path)

    train_x = [x for x in x_d1]
    train_y = [y for y in y_d1]

    return train_x, train_y

def get_data_test(test_image_path):
    x_d1 = img_generator(test_image_path)

    train_x = [x for x in x_d1]

    return train_x

def concat(x1, x2):
    xp = cp.get_array_module(x1, x2)
    return xp.concatenate((x1, x2), axis=1)

def slice2d(x):
    xp = cp.get_array_module(x)
    length = x.shape[0]
    x = x.reshape(length, 28, 28)
    new = []
    for i in range(length):
        new.append(x[i, 3:25, 3:25])
    new = np.array(new)
    print(new.shape)
    return new

def poly(x):
    x = cp.asarray(x)
    x = x.reshape(x.shape[0], 28, 28)
    xt = cp.rot90(x, axes=(1,2))
    return cp.einsum('ijk,ijk->ijk', x, xt)

def conv(x, filter):
    x = cp.asarray(x)
    filter = cp.asarray(filter)
    x = x.reshape(x.shape[0],28,28)
    sub_shape = filter.shape
    view_shape = tuple(np.subtract(x.shape[1:],sub_shape)+1) + sub_shape
    strides = x[0].strides + x[0].strides
    result = []
    for sample in x:
        sub_matrices = stride_tricks.as_strided(sample, view_shape, strides)
        masked = cp.einsum('ijkl,kl->ij', sub_matrices, filter)
        masked = masked.reshape(masked.shape[0]*masked.shape[1]).tolist()
        result.append(masked)
    return result

def centerize(X_):
  ax = int(math.sqrt(len(X_[1])))
  index_mat = cp.array(range(ax))
  X_ = cp.asarray(X_).reshape(-1, ax, ax)
  X = copy.deepcopy(X_)

  bin_threshold = 0.3
  for i in range(X.shape[0]):
    X_[i] = np.where(X_[i] > (255 * bin_threshold), 1, 0)


  Xweight = cp.einsum('ijk, j->i', X_, index_mat)
  Yweight = cp.einsum('ijk, k->i', X_, index_mat)

  X_move = []
  Y_move = []
  for i in range(X.shape[0]):
    X_move.append(int(ax/2) - int(Xweight[i] / cp.sum(X_[i]))) # 왼쪽으로
    Y_move.append(int(ax/2) - int(Yweight[i] / cp.sum(X_[i]))) # 위로
  for i in range(X.shape[0]):
    X[i] = cp.roll(X[i], np.array(X_move[i]), axis = 0)
    X[i] = cp.roll(X[i], np.array(Y_move[i]), axis = 1)
  return X

def reshape2_1D(X):
  print("Input : ",cp.array(X).shape)
  len_ = X.shape[1]
  X = X.reshape(-1, len_ * len_)
  print("Reshaped to 1d : ",cp.array(X).shape)
  return X

def reshape2_2D(X):
  print("Input : ",cp.array(X).shape)
  ax = int(math.sqrt(len(X[1])))
  X = cp.asarray(X).reshape(-1, ax, ax)
  print("Reshaped to 2D : ",cp.array(X).shape)
  return X

def crop_center(img,crop_len):
    img = cp.asnumpy(img)
    y, x = img.shape
    startx = x//2-(crop_len//2)
    starty = y//2-(crop_len//2)    
    return img[starty:starty+crop_len,startx:startx+crop_len]

def crop(img_set, crop_len):
  result = []
  for i in range(img_set.shape[0]):
    result.append(crop_center(img_set[i], crop_len))
  return np.array(result)

In [0]:
X,Y = get_data("./newtrain-images-idx3-ubyte",
               "./newtrain-labels-idx1-ubyte")
X_test = get_data_test("./mnist_new_testall-patterns-idx3-ubyte")

print("Start Data Centerize\n")
X = centerize(X)
X_test = centerize(X_test)

print("Start Data Crop\n")
croplength = 23

X_cropped =  crop(cp.asnumpy(X), croplength)
X = copy.deepcopy(reshape2_1D(X_cropped))
X = cp.asnumpy(X)

X_test_cropped =  crop(cp.asnumpy(X_test), croplength)
X_test = copy.deepcopy(reshape2_1D(X_test_cropped))
X_test = cp.asnumpy(X_test)

print("Start Data PCA & Polynomial\n")
pca = PCA(n_components=0.95)
poly = PolynomialFeatures(2)

X = pca.fit_transform(X)
X = poly.fit_transform(X)
X = cp.asarray(X)

X_test = pca.transform(X_test)
X_test = poly.fit_transform(X_test)
X_test = cp.array(X_test)

print("Start Fitting\n")
clf = SVC(C=1000, eta=0.0008, max_epoch=20, batch_size=256, mth = 30)
y_pred =clf.fit(X,Y).predict(X_test)

print("Start Making prediction text File\n")
with open('./prediction.txt', 'w') as file:
        for i in y_pred:
            file.write('%d\n' % i)